## Simulate the paper of Guizzi (optimal case)

Use the package to make your own LAES plant. A charging and discharge cycle are implemented in the package, but here is how you could use the different components to simulate any LAES plant configuration. Extra compressors can be implemented, the parameters can be changed, etc.

Remarks: 
- I only used Essotherm650 as thermal oil. If you want to change this, the cp values have to be put into the State definition.
- The optimal liquefaction pressure p2 and optimal discharge pressure p2R are used.

In [1]:
# Use the Package 
using LAES

### Charging cycle

In [2]:
# Initialize state10 (ambient)
p10 = 100000 #[Pa]
T10 = 298.15 #[K]
y_N2 = 0.77
x_N2 = 0.0
liquid_fraction = 0.0
mdot1 = 1.0 #[kg/s]
state_initial = State("Air",p10,T10,mdot1;phase = "gas",y_N2 = y_N2,x_N2 = x_N2,liquid_fraction = liquid_fraction)
state_initial.liquid_fraction = 0
state1 = state_initial

# Initialize the thermal oil 
stateOil_in = State("Essotherm650",100000,288.15,0.999)
oil_distribution = [0.485 0.515] #first value = intercooler 2A-2B,...
stateOil_in1 = State(stateOil_in.fluid,stateOil_in.p,stateOil_in.T,stateOil_in.mdot*oil_distribution[1])
stateOil_in2 = State(stateOil_in.fluid,stateOil_in.p,stateOil_in.T,stateOil_in.mdot*oil_distribution[2])

# Initialize the coolants
methanol_min = State("Methanol",100000,214,0.437)
methanol_max = State("Methanol",100000,288,0.437)
propane_min = State("Propane",100000,93,1.019)
propane_max = State("Propane",100000,214,1.019)

# The pressure after the compressors
p2A = 1480000 #[Pa]
p2C = 18098000 #[Pa]

# The Temperature of the compressed air after the coldbox
T4 = 98 #[K] (propane_min.T + pinch)

# Other parameters
pinch_IC = 20 #[K]
pinch_coldbox = 5 #[K]
pressure_loss_IC = 0.01
η_c = 0.85
η_cryo_e = 0.7
solutions = []

for i = 1:20
    global state2A = isentropic_compressor(state1,p2A,η_c)
    global state2B,stateOil_out1 = intercooler("Cool",state2A,stateOil_in1,pinch_IC,pressure_loss_IC)
    global state2C = isentropic_compressor(state2B,p2C,η_c)
    global state2,stateOil_out2 = intercooler("Cool",state2C,stateOil_in2,pinch_IC,pressure_loss_IC)
    state3_p = state2.p-state2.p*pressure_loss_IC
    global state4 = State("Air",state3_p-state3_p*pressure_loss_IC,T4,state2.mdot;phase = state2.phase,y_N2 = state2.y_N2,x_N2 = state2.x_N2,liquid_fraction = state2.liquid_fraction)
    global state5 =  isentropic_cryoexpander(state4,102000,η_cryo_e)
    global state6,state7 = separator(state5)
    # The optimal case, means that there are 2 pinch points of 5K --> The outlet temperature of the cold air can be calculated
    global yield,T9 = pinch_coldbox_optimal(state2,pinch_coldbox,methanol_min,methanol_max,propane_min,propane_max,η_cryo_e,pressure_loss_IC)
    global state9 = State("Air",state1.p,T9,state7.mdot;phase = state7.phase,y_N2 = state7.y_N2,x_N2 = state7.x_N2,liquid_fraction = state7.liquid_fraction)
    global state10 = State("Air",state1.p,T10,state6.mdot;phase = "gas",y_N2 = 0.77,x_N2 = 0,liquid_fraction = state9.liquid_fraction)#standard conditions

    global state1 = State("Air",state1.p,state9.mdot*state9.T+state10.mdot*state10.T,mdot1;phase = "gas",y_N2 = state9.mdot*state9.y_N2+state10.mdot*state10.y_N2,x_N2 = x_N2,liquid_fraction = state10.liquid_fraction)

    #check convergence
    #println(i)
    #println(state1) 

    push!(solutions,state1)
    if length(solutions) > 1 && compare(solutions[end-1],solutions[end])
        break
    end
    if i == 20
        @warn("Did not converge after 20 iterations")
    end
end

state2H = stateOil_in
state1H = State("Essotherm650",stateOil_out1.p,(stateOil_out1.T*oil_distribution[1]+stateOil_out2.T*oil_distribution[2]),(stateOil_out1.mdot+stateOil_out2.mdot))
state3H = state1H

state1,state2A,state2B,state2C,state2,state4,state5,state6,state7,state9,state10,state1H,yield

(LAES.AirState("gas", 100000.0, 296.24978024999996, 298517.5757809664, 6881.228981694648, 1.0, 0.7932693673746807, 0.0, 0.0), LAES.AirState("gas", 1.48e6, 686.8207802499961, 704515.9612441772, 6973.24519864138, 1.0, 0.7932693673746807, 0.0, 0.0), LAES.AirState("gas", 1.4652e6, 308.15, 307095.72532254364, 6137.482316146095, 1.0, 0.7932693673746807, 0.0, 0.0), LAES.AirState("gas", 1.8098e7, 679.0999999999962, 698523.5374569043, 6226.965215734713, 1.0, 0.7932693673746807, 0.0, 0.0), LAES.AirState("gas", 1.791702e7, 308.15, 276969.44859757734, 5325.926400965863, 1.0, 0.7932693673746807, 0.0, 0.0), LAES.AirState("gas", 1.7560471302e7, 98.0, -78703.64414905178, 3306.8589794249174, 1.0, 0.7932693673746807, 0.0, 0.0), LAES.AirState("2phase", 102000.0, 78.99899999999522, -93347.31199772954, 3386.5985991940597, 1.0, 0.9256479423055566, 0.7693857525152417, 0.8504999999999999), LAES.AirState("liquid", 102000.0, 78.99899999999522, -123586.77624124315, 3016.2962086550497, 0.8504999999999999, 0.0, 0.

### Dicharge cycle

In [6]:
# Initialize the thermal oil
stateOil_in = state3H
oil_distribution = [0.357 0.323 0.320] #first value = intercooler 5R-6R,...

# The pressure after the pump
p2R = 6500000

# Other parameters
pinch_coldbox = 5
pinch_superheaters = 10
pressure_loss = 0.01
η_pump = 0.7
η_e = 0.85

# Expander pressures
p3R =(p2R-p2R*pressure_loss)
p4R = (p3R-p3R*pressure_loss)
p5R = (p4R-p4R*pressure_loss)
p6R = (p5R-p5R*pressure_loss)
r = (p6R/101000*(1-pressure_loss)^2)^(1/3) 
p7R = p6R/r
p8R = p7R-p7R*pressure_loss
p9R = p8R/r
p10R = p9R-p9R*pressure_loss
p11R = p10R/r
p12R = p11R-p11R*pressure_loss
expander_pressures = [p7R p9R p11R]

# Discharge calculations
state1R = storage_tank(state6,100000)

state1R,state2R,state3R,state4R,state5R,state6R,state7R,state8R,state9R,state10R,state11R,state12R,state4H = discharge_cycle(state1R,stateOil_in,oil_distribution,propane_max,methanol_max,pinch_coldbox,pinch_superheaters,p2R,expander_pressures,η_e,η_pump,pressure_loss)

(LAES.AirState("liquid", 100000.0, 78.67184703677418, -124193.02196738153, 3008.6317598816368, 0.8504999999999999, 0.0, 0.7693857525152417, 1.0), LAES.AirState("gas", 6.5e6, 81.68484703677429, -114997.26564351506, 3043.054573287856, 0.8504999999999999, 0.7693857525152417, 0.0, 0.0), LAES.AirState("gas", 6.435e6, 209.0, 176398.77009590165, 5214.086329116974, 0.8504999999999999, 0.7693857525152417, 0.0, 0.0), LAES.AirState("gas", 6.37065e6, 283.0, 266407.53860083903, 5586.918214159231, 0.8504999999999999, 0.7693857525152417, 0.0, 0.0), LAES.AirState("gas", 6.3069435e6, 434.5689999999988, 431893.3057555226, 6059.148273655015, 0.8504999999999999, 0.7693857525152417, 0.0, 0.0), LAES.AirState("gas", 6.243874065e6, 615.2111950000001, 625256.8656633586, 6434.059746030945, 0.8504999999999999, 0.7693857525152417, 0.0, 0.0), LAES.AirState("gas", 1.589716754989629e6, 450.12119499999875, 452507.80877440074, 6504.567244942205, 0.8504999999999999, 0.7693857525152417, 0.0, 0.0), LAES.AirState("gas", 1

### Calculate the efficiency

In [7]:
W_charging = (state2A.h-state1.h) + (state2C.h-state2B.h) - (state4.h-state5.h) #compressors minus cryoexpander
W_discharging = (state6R.h-state7R.h) + (state8R.h-state9R.h) + (state10R.h-state11R.h) - (state2R.h-state1R.h) #expanders minus cryopump
η_RT = yield*(W_discharging/W_charging)

0.5501522540342919